<a href="https://colab.research.google.com/github/acram002/AI-Driven-Recipe-Suggestion-System/blob/main/spoonacularAPI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
5bf6609405mshe61b903936f23cep1fd1b6jsn1ecaaf102007

In [ ]:
# random
import http.client
import json
import pandas as pd
import os

# Spoonacular API and RapidAPI Configuration
RAPIDAPI_KEY = "5bf6609405mshe61b903936f23cep1fd1b6jsn1ecaaf102007"  # Replace with your RapidAPI key

# Function to query Spoonacular API for recipes
def get_recipes(query, diet, intolerances, include_ingredients, exclude_ingredients, num_recipes=100):
    conn = http.client.HTTPSConnection("spoonacular-recipe-food-nutrition-v1.p.rapidapi.com")

    headers = {
        'x-rapidapi-key': RAPIDAPI_KEY,
        'x-rapidapi-host': "spoonacular-recipe-food-nutrition-v1.p.rapidapi.com"
    }

    # Pagination setup: start with the first page
    offset = 0
    all_recipes = []

    while len(all_recipes) < num_recipes:
        # API endpoint parameters
        url = f"/recipes/complexSearch?query={query}&diet={diet}&intolerances={intolerances}&includeIngredients={include_ingredients}&excludeIngredients={exclude_ingredients}&instructionsRequired=true&addRecipeInformation=true&number={num_recipes}"

        conn.request("GET", url, headers=headers)
        res = conn.getresponse()
        data = res.read()

        recipes_data = json.loads(data.decode("utf-8"))

        # If no recipes found, break the loop
        if "results" not in recipes_data or not recipes_data["results"]:
            break

        all_recipes.extend(recipes_data["results"])
        offset += 10  # Increment the offset to get the next batch of results

        # Break the loop if we have enough recipes
        if len(all_recipes) >= num_recipes:
            break

    return all_recipes[:num_recipes]  # Limit to the requested number of recipes

# Function to process and store recipe data
def process_recipes(recipes_data, allergies, dietary_preferences):
    recipe_list = []

    for recipe in recipes_data:
        title = recipe["title"]

        # Extract ingredients and instructions
        ingredients = []
        for ingredient in recipe.get("extendedIngredients", []):
            ingredients.append(ingredient["name"])

        # Check if instructions are available
        instructions = recipe.get("instructions", "No instructions provided")

        # Ensure allergies and dietary preferences are included, even if missing in data
        recipe_allergies = allergies if allergies else "Not specified"
        recipe_dietary_preferences = dietary_preferences if dietary_preferences else "Not specified"

        # Store the recipe data
        recipe_list.append({
            "title": title,
            "ingredients": ingredients if ingredients else ["No ingredients provided"],
            "instructions": instructions,
            "allergies": recipe_allergies,
            "dietary_preferences": recipe_dietary_preferences
        })

    return recipe_list

# Define the input parameters
ingredient_query = "salad"  # Example query
dietary_preferences = "vegetarian"  # Example dietary preference (can also be "vegan", "gluten free", etc.)
allergies = "gluten"  # Example allergens (e.g., "gluten", "dairy", "peanut")
include_ingredients = "cheese,nuts"  # Ingredients that must be included
exclude_ingredients = "eggs"  # Ingredients that must be excluded

# Fetch recipes from Spoonacular API
recipes_data = get_recipes(ingredient_query, dietary_preferences, allergies, include_ingredients, exclude_ingredients, num_recipes=10)

# Process the recipes data
if recipes_data:
    recipes = process_recipes(recipes_data, allergies, dietary_preferences)

    # Convert the list to a DataFrame
    df = pd.DataFrame(recipes)

    # Ensure the directory exists before saving the file
    save_path = "/content/drive/MyDrive/spoonacular_recipes_dataset.csv"
    os.makedirs(os.path.dirname(save_path), exist_ok=True)

    # Save dataset to a CSV file
    df.to_csv(save_path, index=False)
    print(f"Dataset created with {len(df)} recipes.")
else:
    print("No recipes found or there was an error with the API request.")


In [ ]:
# All columns
import http.client
import json
import pandas as pd

# Spoonacular API and RapidAPI Configuration
RAPIDAPI_KEY = "5bf6609405mshe61b903936f23cep1fd1b6jsn1ecaaf102007"  # Replace with your RapidAPI key

# Function to query Spoonacular API for recipes
def get_recipes(query, diet, intolerances, include_ingredients, exclude_ingredients, num_recipes=5):
    conn = http.client.HTTPSConnection("spoonacular-recipe-food-nutrition-v1.p.rapidapi.com")

    headers = {
        'x-rapidapi-key': RAPIDAPI_KEY,
        'x-rapidapi-host': "spoonacular-recipe-food-nutrition-v1.p.rapidapi.com"
    }

    # API endpoint parameters
    url = f"/recipes/complexSearch?query={query}&diet={diet}&intolerances={intolerances}&includeIngredients={include_ingredients}&excludeIngredients={exclude_ingredients}&instructionsRequired=true&addRecipeInformation=true&number={num_recipes}"

    conn.request("GET", url, headers=headers)

    res = conn.getresponse()
    data = res.read()

    return json.loads(data.decode("utf-8"))

# Function to process and store recipe data
def process_recipes(recipes_data, allergies, dietary_preferences):
    recipe_list = []

    for recipe in recipes_data["results"]:
        title = recipe["title"]

        # Extract ingredients and instructions
        ingredients = []
        for ingredient in recipe.get("analyzedInstructions", []):
            for step in ingredient.get("steps", []):
                step_ingredients = [ingredient["name"] for ingredient in step.get("ingredients", [])]
                ingredients.extend(step_ingredients)

        instructions = []
        for instruction in recipe.get("analyzedInstructions", []):
            for step in instruction.get("steps", []):
                instructions.append(step.get("step", "No instructions provided"))

        # Store the recipe data
        recipe_list.append({
            "title": title,
            "ingredients": ingredients,
            "instructions": " ".join(instructions) if instructions else "No instructions provided",
            "allergies": allergies,
            "dietary_preferences": dietary_preferences
        })

    return recipe_list
# EDIT THESE
# Define the input parameters
ingredient_query = ""  # Example query
dietary_preferences = "vegetarian"  # Example dietary preference (can also be "vegan", "gluten free", etc.)
allergies = "gluten"  # Example allergens (e.g., "gluten", "dairy", "peanut")
include_ingredients = "cheese,nuts"  # Ingredients that must be included
exclude_ingredients = "eggs"  # Ingredients that must be excluded

# Fetch recipes from Spoonacular API
recipes_data = get_recipes(ingredient_query, dietary_preferences, allergies, include_ingredients, exclude_ingredients, num_recipes=5)

# Process the recipes data
if recipes_data:
    recipes = process_recipes(recipes_data, allergies, dietary_preferences)

    # Convert the list to a DataFrame
    df = pd.DataFrame(recipes)

    # Save dataset to a CSV file
    df.to_csv("spoonacular_recipes_dataset1.csv", index=False)
    print(f"Dataset created with {len(df)} recipes.")
else:
    print("No recipes found or there was an error with the API request.")


In [ ]:
import http.client
import json
import pandas as pd
import os
import random
from urllib.parse import quote_plus

# Spoonacular API and RapidAPI Configuration
RAPIDAPI_KEY = "5bf6609405mshe61b903936f23cep1fd1b6jsn1ecaaf102007"  # Replace with your RapidAPI key

# Function to query Spoonacular API for recipes
def get_recipes(query, diet, intolerances, include_ingredients, exclude_ingredients, num_recipes=5):
    conn = http.client.HTTPSConnection("spoonacular-recipe-food-nutrition-v1.p.rapidapi.com")

    headers = {
        'x-rapidapi-key': RAPIDAPI_KEY,
        'x-rapidapi-host': "spoonacular-recipe-food-nutrition-v1.p.rapidapi.com"
    }

    # URL encoding to handle spaces and special characters
    query = quote_plus(query)
    diet = quote_plus(diet)
    intolerances = quote_plus(intolerances)
    include_ingredients = quote_plus(include_ingredients)
    exclude_ingredients = quote_plus(exclude_ingredients)

    # Pagination setup: start with the first page
    offset = 0
    all_recipes = []

    while len(all_recipes) < num_recipes:
        # API endpoint parameters
        url = f"/recipes/complexSearch?query={query}&diet={diet}&intolerances={intolerances}&includeIngredients={include_ingredients}&excludeIngredients={exclude_ingredients}&instructionsRequired=true&addRecipeInformation=true&number=10&offset={offset}"

        conn.request("GET", url, headers=headers)
        res = conn.getresponse()
        data = res.read()

        recipes_data = json.loads(data.decode("utf-8"))

        # If no recipes found, break the loop
        if "results" not in recipes_data or not recipes_data["results"]:
            break

        all_recipes.extend(recipes_data["results"])
        offset += 10  # Increment the offset to get the next batch of results

        # Break the loop if we have enough recipes
        if len(all_recipes) >= num_recipes:
            break

    return all_recipes[:num_recipes]  # Limit to the requested number of recipes

# Function to process and store recipe data
def process_recipes(recipes_data, allergies, dietary_preferences):
    recipe_list = []

    for recipe in recipes_data:
        title = recipe["title"]

        # Extract ingredients
        ingredients = []
        if "extendedIngredients" in recipe:
            for ingredient in recipe["extendedIngredients"]:
                ingredients.append(ingredient["name"])
        else:
            ingredients = ["No ingredients provided"]

        # Extract instructions
        instructions = recipe.get("instructions", "No instructions provided")

        # Ensure allergies and dietary preferences are included
        recipe_allergies = allergies if allergies else "Not specified"
        recipe_dietary_preferences = dietary_preferences if dietary_preferences else "Not specified"

        # Store the recipe data
        recipe_list.append({
            "title": title,
            "ingredients": ingredients if ingredients else ["No ingredients provided"],
            "instructions": instructions,
            "allergies": recipe_allergies,
            "dietary_preferences": recipe_dietary_preferences
        })

    return recipe_list

# Randomize the input parameters
ingredient_queries = ["salad", "soup", "pasta", "dessert", "smoothie"]
dietary_preferences_list = ["vegetarian", "vegan", "gluten free", "paleo", "keto"]
allergies_list = ["gluten", "dairy", "peanut", "soy", "egg", "tree nut"]
include_ingredients_list = ["cheese", "nuts", "tomato", "spinach", "chicken", "carrot"]
exclude_ingredients_list = ["eggs", "milk", "wheat", "peanut", "soy", "fish"]

# Randomly select one option from each list
ingredient_query = random.choice(ingredient_queries)
dietary_preferences = random.choice(dietary_preferences_list)
allergies = random.choice(allergies_list)
include_ingredients = random.choice(include_ingredients_list)
exclude_ingredients = random.choice(exclude_ingredients_list)

# Optionally, you can randomize multiple items from each list
num_ingredients = 2  # Example: randomly pick 2 ingredients from the list
include_ingredients = ','.join(random.sample(include_ingredients_list, num_ingredients))
exclude_ingredients = ','.join(random.sample(exclude_ingredients_list, num_ingredients))

# Output the randomized inputs (you can remove this print statement if not needed)
print(f"Ingredient Query: {ingredient_query}")
print(f"Dietary Preferences: {dietary_preferences}")
print(f"Allergies: {allergies}")
print(f"Include Ingredients: {include_ingredients}")
print(f"Exclude Ingredients: {exclude_ingredients}")

# Fetch recipes from Spoonacular API using randomized inputs
recipes_data = get_recipes(ingredient_query, dietary_preferences, allergies, include_ingredients, exclude_ingredients, num_recipes=5)

# Process the recipes data
if recipes_data:
    recipes = process_recipes(recipes_data, allergies, dietary_preferences)

    # Convert the list to a DataFrame
    df = pd.DataFrame(recipes)

    # Ensure the directory exists before saving the file
    save_path = "/content/drive/MyDrive/spoonacular_recipes_dataset.csv"
    os.makedirs(os.path.dirname(save_path), exist_ok=True)

    # Save dataset to a CSV file
    df.to_csv(save_path, index=False)
    print(f"Dataset created with {len(df)} recipes.")
else:
    print("No recipes found or there was an error with the API request.")


In [ ]:
#good
import http.client
import json
import pandas as pd
import os

# Spoonacular API and RapidAPI Configuration
RAPIDAPI_KEY = "5bf6609405mshe61b903936f23cep1fd1b6jsn1ecaaf102007"  # Replace with your RapidAPI key

# Function to query Spoonacular API for recipes
def get_recipes(diet, intolerances, num_recipes=5):
    conn = http.client.HTTPSConnection("spoonacular-recipe-food-nutrition-v1.p.rapidapi.com")

    headers = {
        'x-rapidapi-key': RAPIDAPI_KEY,
        'x-rapidapi-host': "spoonacular-recipe-food-nutrition-v1.p.rapidapi.com"
    }

    # API endpoint parameters
    #url = f"/recipes/complexSearch?diet={diet}&intolerances={intolerances}fillIngredients=true&number={num_recipes}"
    url = f"/recipes/complexSearch?diet={diet}&intolerances={intolerances}&instructionsRequired=true&fillIngredients=true&addRecipeInformation=true&number={num_recipes}"

    conn.request("GET", url, headers=headers)
    res = conn.getresponse()
    data = res.read()

    return json.loads(data.decode("utf-8"))

# Function to process and store recipe data
def process_recipes(recipes_data, allergies, dietary_preferences):
    recipe_list = []

    for recipe in recipes_data["results"]:
        title = recipe["title"]

        # Extract ingredients and instructions
        ingredients = []
        if "extendedIngredients" in recipe:
            for ingredient in recipe.get("extendedIngredients", []):
                ingredients.append(ingredient["name"])

        instructions = recipe.get("instructions", "No instructions provided")

        # Dietary preferences
        diet = ', '.join(recipe.get("diets", [])) if "diets" in recipe else "Not specified"

        # Allergies/intolerances
        intolerances = ', '.join(recipe.get("intolerances", [])) if "intolerances" in recipe else "Not specified"

        # Store the recipe data
        recipe_list.append({
            "title": title,
            "ingredients": ingredients if ingredients else ["No ingredients provided"],
            "instructions": instructions,
            "allergies": intolerances,
            "dietary_preferences": diet
        })

    return recipe_list

# Define the input parameters
ingredient_query = "salad"  # Example query
dietary_preferences = "vegetarian"  # Example dietary preference (can also be "vegan", "gluten free", etc.)
allergies = "Gluten"  # Example allergens (e.g., "gluten", "dairy", "peanut")
include_ingredients = "cheese,nuts"  # Ingredients that must be included
exclude_ingredients = "eggs"  # Ingredients that must be excluded

# Fetch recipes from Spoonacular API
recipes_data = get_recipes(dietary_preferences, allergies, num_recipes=5)

# Process the recipes data
if recipes_data:
    recipes = process_recipes(recipes_data, allergies, dietary_preferences)

    # Convert the list to a DataFrame
    df = pd.DataFrame(recipes)

    # Ensure the directory exists before saving the file
    save_path = "/content/drive/MyDrive/spoonacular_recipes_dataset.csv"
    os.makedirs(os.path.dirname(save_path), exist_ok=True)

    # Save dataset to a CSV file
    df.to_csv(save_path, index=False)
    print(f"Dataset created with {len(df)} recipes.")
else:
    print("No recipes found or there was an error with the API request.")


In [ ]:
import http.client
import json
import pandas as pd
import os

# Spoonacular API and RapidAPI Configuration
RAPIDAPI_KEY = "5bf6609405mshe61b903936f23cep1fd1b6jsn1ecaaf102007"  # Replace with your RapidAPI key

# Function to query Spoonacular API for recipes
def get_recipes(diet, intolerances, num_recipes=5):
    conn = http.client.HTTPSConnection("spoonacular-recipe-food-nutrition-v1.p.rapidapi.com")

    headers = {
        'x-rapidapi-key': RAPIDAPI_KEY,
        'x-rapidapi-host': "spoonacular-recipe-food-nutrition-v1.p.rapidapi.com"
    }

    # API endpoint parameters
    url = f"/recipes/complexSearch?diet={diet}&intolerances={intolerances}&instructionsRequired=true&fillIngredients=true&addRecipeInformation=true&number={num_recipes}"

    conn.request("GET", url, headers=headers)
    res = conn.getresponse()
    data = res.read()

    return json.loads(data.decode("utf-8"))

# Function to process and store recipe data
def process_recipes(recipes_data, allergies, dietary_preferences):
    recipe_list = []

    for recipe in recipes_data["results"]:
        title = recipe["title"]

        # Extract ingredients
        ingredients = []
        if "extendedIngredients" in recipe:
            for ingredient in recipe.get("extendedIngredients", []):
                ingredients.append(ingredient["name"])

        # Allergies/intolerances
        intolerances = ', '.join(recipe.get("intolerances", [])) if "intolerances" in recipe else "Not specified"

        # Dietary preferences
        diet = ', '.join(recipe.get("diets", [])) if "diets" in recipe else "Not specified"

        # Store the recipe data
        recipe_list.append({
            "title": title,
            "ingredients": ingredients if ingredients else ["No ingredients provided"],
            "allergies": intolerances,
            "dietary_preferences": diet
        })

    return recipe_list

# Define the input parameters
ingredient_query = ""  # Example query
dietary_preferences = ""  # Example dietary preference (can also be "vegan", "gluten free", etc.)
allergies = ""  # Example allergens (e.g., "gluten", "dairy", "peanut")
include_ingredients = ""  # Ingredients that must be included
exclude_ingredients = ""  # Ingredients that must be excluded

# Fetch recipes from Spoonacular API
recipes_data = get_recipes(dietary_preferences, allergies, num_recipes=5)

# Process the recipes data
if recipes_data:
    recipes = process_recipes(recipes_data, allergies, dietary_preferences)

    # Convert the list to a DataFrame
    df = pd.DataFrame(recipes)

    # Ensure the directory exists before saving the file
    save_path = "/content/drive/MyDrive/spoonacular_recipes_dataset.csv"
    os.makedirs(os.path.dirname(save_path), exist_ok=True)

    # Save dataset to a CSV file
    df.to_csv(save_path, index=False)
    print(f"Dataset created with {len(df)} recipes.")
else:
    print("No recipes found or there was an error with the API request.")


In [ ]:
import http.client
import json
import pandas as pd
import os

# Spoonacular API and RapidAPI Configuration
RAPIDAPI_KEY = "5bf6609405mshe61b903936f23cep1fd1b6jsn1ecaaf102007"  # Replace with your key

# Function to fetch random recipes
def get_random_recipes(num_recipes=5, tags=""):
    conn = http.client.HTTPSConnection("spoonacular-recipe-food-nutrition-v1.p.rapidapi.com")
    headers = {
        'x-rapidapi-key': RAPIDAPI_KEY,
        'x-rapidapi-host': "spoonacular-recipe-food-nutrition-v1.p.rapidapi.com"
    }
    url = f"/recipes/random?number={num_recipes}&tags={tags}"
    conn.request("GET", url, headers=headers)
    res = conn.getresponse()
    data = res.read()
    return json.loads(data.decode("utf-8"))

# Function to fetch full details for a given recipe ID
def get_recipe_details(recipe_id):
    conn = http.client.HTTPSConnection("spoonacular-recipe-food-nutrition-v1.p.rapidapi.com")
    headers = {
        'x-rapidapi-key': RAPIDAPI_KEY,
        'x-rapidapi-host': "spoonacular-recipe-food-nutrition-v1.p.rapidapi.com"
    }
    url = f"/recipes/{recipe_id}/information?includeNutrition=false"
    conn.request("GET", url, headers=headers)
    res = conn.getresponse()
    data = res.read()
    return json.loads(data.decode("utf-8"))

# Function to process recipe data and extract relevant info
def process_recipes(recipes_data):
    recipe_list = []

    for recipe in recipes_data["recipes"]:  # random endpoint returns "recipes"
        recipe_id = recipe["id"]
        details = get_recipe_details(recipe_id)

        title = details.get("title", "No title")
        ingredients = [ing["name"] for ing in details.get("extendedIngredients", [])]

        # Allergy/intolerance flags
        allergies = []
        if details.get("glutenFree"):
            allergies.append("gluten free")
        if details.get("dairyFree"):
            allergies.append("dairy free")
        if details.get("vegan"):
            allergies.append("vegan")
        if details.get("vegetarian"):
            allergies.append("vegetarian")

        # Diet info
        diet = ', '.join(details.get("diets", [])) if "diets" in details else "Not specified"

        recipe_list.append({
            "title": title,
            "ingredients": ingredients if ingredients else ["No ingredients provided"],
            "allergies": ', '.join(allergies) if allergies else "Not specified",
            "dietary_preferences": diet
        })

    return recipe_list

# -----------------------
# CONFIGURATION
# -----------------------
num_recipes = 5
tags = ""  # e.g., "vegetarian,gluten free" or leave blank for fully random

# Get random recipes
recipes_data = get_random_recipes(num_recipes=num_recipes, tags=tags)

# Process and store them
if recipes_data:
    recipes = process_recipes(recipes_data)
    df = pd.DataFrame(recipes)

    # Save to Google Drive or local path
    save_path = "/content/drive/MyDrive/spoonacular_recipes_dataset.csv"
    os.makedirs(os.path.dirname(save_path), exist_ok=True)
    df.to_csv(save_path, index=False)
    print(f"✅ Dataset created with {len(df)} recipes. Saved to: {save_path}")
else:
    print("❌ No recipes found or API request failed.")


In [ ]:
import http.client
import json
import pandas as pd
import os
import time

# Spoonacular API and RapidAPI Configuration
RAPIDAPI_KEY = "5bf6609405mshe61b903936f23cep1fd1b6jsn1ecaaf102007"

# Function to fetch random recipes
def get_random_recipes(num_recipes=10, tags=""):
    conn = http.client.HTTPSConnection("spoonacular-recipe-food-nutrition-v1.p.rapidapi.com")
    headers = {
        'x-rapidapi-key': RAPIDAPI_KEY,
        'x-rapidapi-host': "spoonacular-recipe-food-nutrition-v1.p.rapidapi.com"
    }
    url = f"/recipes/random?number={num_recipes}&tags={tags}"
    conn.request("GET", url, headers=headers)
    res = conn.getresponse()
    data = res.read()
    return json.loads(data.decode("utf-8"))

# Function to fetch detailed info for one recipe
def get_recipe_details(recipe_id):
    conn = http.client.HTTPSConnection("spoonacular-recipe-food-nutrition-v1.p.rapidapi.com")
    headers = {
        'x-rapidapi-key': RAPIDAPI_KEY,
        'x-rapidapi-host': "spoonacular-recipe-food-nutrition-v1.p.rapidapi.com"
    }
    url = f"/recipes/{recipe_id}/information?includeNutrition=false"
    conn.request("GET", url, headers=headers)
    res = conn.getresponse()
    data = res.read()
    return json.loads(data.decode("utf-8"))

# Process a batch of recipes
def process_recipes(recipes_data):
    recipe_list = []

    for recipe in recipes_data["recipes"]:
        recipe_id = recipe["id"]
        details = get_recipe_details(recipe_id)
        time.sleep(0.5)  # Prevent throttling

        title = details.get("title", "No title")
        ingredients = [ing["name"] for ing in details.get("extendedIngredients", [])]

        allergies = []
        if details.get("glutenFree"):
            allergies.append("gluten free")
        if details.get("dairyFree"):
            allergies.append("dairy free")

        diet = ', '.join(details.get("diets", [])) if "diets" in details else "Not specified"

        recipe_list.append({
            "title": title,
            "ingredients": ingredients if ingredients else ["No ingredients provided"],
            "allergies": ', '.join(allergies) if allergies else "Not specified",
            "dietary_preferences": diet
        })

    return recipe_list

# -----------------------
# CONFIGURATION
# -----------------------
total_recipes = 1000
batch_size = 10
save_path = "/content/drive/MyDrive/spoonacular_recipes_dataset.csv"
tags = ""

# Ensure save path exists
os.makedirs(os.path.dirname(save_path), exist_ok=True)

# Check if we're resuming
if os.path.exists(save_path):
    existing_df = pd.read_csv(save_path)
    already_fetched = len(existing_df)
    print(f"🔁 Resuming: {already_fetched} recipes already fetched.")
else:
    existing_df = pd.DataFrame()
    already_fetched = 0

# Start fetching remaining recipes
for batch_num in range(already_fetched // batch_size, total_recipes // batch_size):
    print(f"📦 Fetching batch {batch_num + 1} of {total_recipes // batch_size}")
    try:
        recipes_data = get_random_recipes(num_recipes=batch_size, tags=tags)
        if recipes_data:
            processed = process_recipes(recipes_data)
            batch_df = pd.DataFrame(processed)
            batch_df.to_csv(save_path, mode='a', index=False, header=not os.path.exists(save_path) and batch_num == 0)
            print(f"✅ Batch {batch_num + 1} saved.")
        else:
            print("⚠️ Failed to fetch a batch. Skipping...")
        time.sleep(1)
    except Exception as e:
        print(f"❌ Error in batch {batch_num + 1}: {e}")
        break

print(f"\n🎉 Completed. Check {save_path} for your full dataset.")


In [ ]:
import http.client
import json
import pandas as pd
import os
import time
from pathlib import Path

# Spoonacular API and RapidAPI Configuration
RAPIDAPI_KEY = "5bf6609405mshe61b903936f23cep1fd1b6jsn1ecaaf102007"

# Function to fetch random recipes
def get_random_recipes(num_recipes=10, tags=""):
    conn = http.client.HTTPSConnection("spoonacular-recipe-food-nutrition-v1.p.rapidapi.com")
    headers = {
        'x-rapidapi-key': RAPIDAPI_KEY,
        'x-rapidapi-host': "spoonacular-recipe-food-nutrition-v1.p.rapidapi.com"
    }
    url = f"/recipes/random?number={num_recipes}&tags={tags}"
    conn.request("GET", url, headers=headers)
    res = conn.getresponse()
    data = res.read()
    return json.loads(data.decode("utf-8"))

# Function to fetch detailed info for one recipe
def get_recipe_details(recipe_id):
    conn = http.client.HTTPSConnection("spoonacular-recipe-food-nutrition-v1.p.rapidapi.com")
    headers = {
        'x-rapidapi-key': RAPIDAPI_KEY,
        'x-rapidapi-host': "spoonacular-recipe-food-nutrition-v1.p.rapidapi.com"
    }
    url = f"/recipes/{recipe_id}/information?includeNutrition=false"
    conn.request("GET", url, headers=headers)
    res = conn.getresponse()
    data = res.read()
    return json.loads(data.decode("utf-8"))

# Process a batch of recipes
def process_recipes(recipes_data):
    recipe_list = []

    for recipe in recipes_data["recipes"]:
        recipe_id = recipe["id"]
        details = get_recipe_details(recipe_id)
        time.sleep(0.5)

        title = details.get("title", "No title")
        ingredients = [ing["name"] for ing in details.get("extendedIngredients", [])]

        allergies = []
        if details.get("glutenFree"):
            allergies.append("gluten free")
        if details.get("dairyFree"):
            allergies.append("dairy free")

        diet = ', '.join(details.get("diets", [])) if "diets" in details else "Not specified"

        recipe_list.append({
            "title": title,
            "ingredients": ingredients if ingredients else ["No ingredients provided"],
            "allergies": ', '.join(allergies) if allergies else "Not specified",
            "dietary_preferences": diet
        })

    return recipe_list

# -----------------------
# CONFIGURATION
# -----------------------
total_recipes = 1000
batch_size = 10
save_path = "/content/drive/MyDrive/spoonacular_recipes_dataset.csv"
tags = ""

# Ensure path exists
os.makedirs(os.path.dirname(save_path), exist_ok=True)

# Delete old file if exists
if os.path.exists(save_path):
    os.remove(save_path)
    print("🗑️ Old dataset deleted.")

# Start collection
for batch_num in range(total_recipes // batch_size):
    print(f"📦 Fetching batch {batch_num + 1} of {total_recipes // batch_size}")
    try:
        recipes_data = get_random_recipes(num_recipes=batch_size, tags=tags)
        if recipes_data:
            processed = process_recipes(recipes_data)
            batch_df = pd.DataFrame(processed)

            if batch_num == 0:
                batch_df.to_csv(save_path, mode='w', index=False, header=True)
            else:
                batch_df.to_csv(save_path, mode='a', index=False, header=False)

            print(f"✅ Batch {batch_num + 1} saved.")
        else:
            print("⚠️ Empty batch received.")
        time.sleep(1)
    except Exception as e:
        print(f"❌ Error in batch {batch_num + 1}: {e}")
        break

# Final file check
if Path(save_path).exists():
    file_size = Path(save_path).stat().st_size / 1024
    print(f"\n🎉 Done. File saved to: {save_path}")
    print(f"📁 File size: {file_size:.1f} KB")
else:
    print("❌ File was not saved.")


In [ ]:
# includes instructions
import http.client
import json
import pandas as pd
import os
import time
from pathlib import Path
# ✅ STEP 2: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Spoonacular API and RapidAPI Configuration
RAPIDAPI_KEY = "5bf6609405mshe61b903936f23cep1fd1b6jsn1ecaaf102007"

# Function to fetch random recipes
def get_random_recipes(num_recipes=10, tags=""):
    conn = http.client.HTTPSConnection("spoonacular-recipe-food-nutrition-v1.p.rapidapi.com")
    headers = {
        'x-rapidapi-key': RAPIDAPI_KEY,
        'x-rapidapi-host': "spoonacular-recipe-food-nutrition-v1.p.rapidapi.com"
    }
    url = f"/recipes/random?number={num_recipes}&tags={tags}"
    conn.request("GET", url, headers=headers)
    res = conn.getresponse()
    data = res.read()
    return json.loads(data.decode("utf-8"))

# Function to fetch detailed info for one recipe
def get_recipe_details(recipe_id):
    conn = http.client.HTTPSConnection("spoonacular-recipe-food-nutrition-v1.p.rapidapi.com")
    headers = {
        'x-rapidapi-key': RAPIDAPI_KEY,
        'x-rapidapi-host': "spoonacular-recipe-food-nutrition-v1.p.rapidapi.com"
    }
    url = f"/recipes/{recipe_id}/information?includeNutrition=false"
    conn.request("GET", url, headers=headers)
    res = conn.getresponse()
    data = res.read()
    return json.loads(data.decode("utf-8"))

# Process a batch of recipes
def process_recipes(recipes_data):
    recipe_list = []

    for recipe in recipes_data["recipes"]:
        recipe_id = recipe["id"]
        details = get_recipe_details(recipe_id)
        time.sleep(0.5)

        title = details.get("title", "No title")
        ingredients = [ing["name"] for ing in details.get("extendedIngredients", [])]

        # Extract instructions (if available)
        instructions = []
        for section in details.get("analyzedInstructions", []):
            for step in section.get("steps", []):
                instructions.append(step.get("step"))

        instructions_text = "\n".join(instructions).strip() if instructions else "No instructions provided."

        allergies = []
        if details.get("glutenFree"):
            allergies.append("gluten free")
        if details.get("dairyFree"):
            allergies.append("dairy free")

        diet = ', '.join(details.get("diets", [])) if "diets" in details else "Not specified"

        recipe_list.append({
            "title": title,
            "ingredients": ingredients if ingredients else ["No ingredients provided"],
            "instructions": instructions_text,
            "allergies": ', '.join(allergies) if allergies else "Not specified",
            "dietary_preferences": diet
        })

    return recipe_list

# -----------------------
# CONFIGURATION
# -----------------------
total_recipes = 1500
batch_size = 10
save_path = "/content/drive/MyDrive/spoonacular_recipes_dataset1.csv"
tags = ""

# Ensure path exists
os.makedirs(os.path.dirname(save_path), exist_ok=True)

# Delete old file if exists
if os.path.exists(save_path):
    os.remove(save_path)
    print("🗑️ Old dataset deleted.")

# Start collection
for batch_num in range(total_recipes // batch_size):
    print(f"📦 Fetching batch {batch_num + 1} of {total_recipes // batch_size}")
    try:
        recipes_data = get_random_recipes(num_recipes=batch_size, tags=tags)
        if recipes_data:
            processed = process_recipes(recipes_data)
            batch_df = pd.DataFrame(processed)

            if batch_num == 0:
                batch_df.to_csv(save_path, mode='w', index=False, header=True)
            else:
                batch_df.to_csv(save_path, mode='a', index=False, header=False)

            print(f"✅ Batch {batch_num + 1} saved.")
        else:
            print("⚠️ Empty batch received.")
        time.sleep(1)
    except Exception as e:
        print(f"❌ Error in batch {batch_num + 1}: {e}")
        break

# Final file check
if Path(save_path).exists():
    file_size = Path(save_path).stat().st_size / 1024
    print(f"\n🎉 Done. File saved to: {save_path}")
    print(f"📁 File size: {file_size:.1f} KB")
else:
    print("❌ File was not saved.")



In [ ]:
import http.client
import json
import pandas as pd
import os
import time
from pathlib import Path

# Spoonacular API and RapidAPI Configuration
RAPIDAPI_KEY = "5bf6609405mshe61b903936f23cep1fd1b6jsn1ecaaf102007"

# Function to fetch random recipes
def get_random_recipes(num_recipes=10, tags=""):
    conn = http.client.HTTPSConnection("spoonacular-recipe-food-nutrition-v1.p.rapidapi.com")
    headers = {
        'x-rapidapi-key': RAPIDAPI_KEY,
        'x-rapidapi-host': "spoonacular-recipe-food-nutrition-v1.p.rapidapi.com"
    }
    url = f"/recipes/random?number={num_recipes}&tags={tags}"
    conn.request("GET", url, headers=headers)
    res = conn.getresponse()
    data = res.read()
    return json.loads(data.decode("utf-8"))

# Function to fetch detailed info for one recipe
def get_recipe_details(recipe_id):
    conn = http.client.HTTPSConnection("spoonacular-recipe-food-nutrition-v1.p.rapidapi.com")
    headers = {
        'x-rapidapi-key': RAPIDAPI_KEY,
        'x-rapidapi-host': "spoonacular-recipe-food-nutrition-v1.p.rapidapi.com"
    }
    url = f"/recipes/{recipe_id}/information?includeNutrition=false"
    conn.request("GET", url, headers=headers)
    res = conn.getresponse()
    data = res.read()
    return json.loads(data.decode("utf-8"))

# Function to clean instructions
def clean_instructions(text):
    if not text:
        return "Instructions not available."
    cutoff_phrases = [
        "Note:", "Nutritional", "Source:", "Enjoy!",
        "provided by", "Food Network", "FN chefs", "do not include personal"
    ]
    for phrase in cutoff_phrases:
        idx = text.lower().find(phrase.lower())
        if idx != -1:
            return text[:idx].strip()
    return text.strip()

# Process a batch of recipes
def process_recipes(recipes_data):
    recipe_list = []

    for recipe in recipes_data["recipes"]:
        recipe_id = recipe["id"]
        details = get_recipe_details(recipe_id)
        time.sleep(0.5)

        title = details.get("title", "No title")
        ingredients = [ing["name"] for ing in details.get("extendedIngredients", [])]

        # Extract and clean instructions
        raw_steps = []
        for section in details.get("analyzedInstructions", []):
            for step in section.get("steps", []):
                raw_steps.append(step.get("step"))

        full_instructions = "\n".join(raw_steps).strip()
        cleaned_instructions = clean_instructions(full_instructions)

        allergies = []
        if details.get("glutenFree"):
            allergies.append("gluten free")
        if details.get("dairyFree"):
            allergies.append("dairy free")

        diet = ', '.join(details.get("diets", [])) if "diets" in details else "Not specified"

        recipe_list.append({
            "title": title,
            "ingredients": ingredients if ingredients else ["No ingredients provided"],
            "instructions": cleaned_instructions,
            "allergies": ', '.join(allergies) if allergies else "Not specified",
            "dietary_preferences": diet
        })

    return recipe_list

# -----------------------
# CONFIGURATION
# -----------------------
total_recipes = 1000
batch_size = 10
save_path = "/content/drive/MyDrive/spoonacular_recipes_dataset2.csv"
tags = ""

# Ensure path exists
os.makedirs(os.path.dirname(save_path), exist_ok=True)

# Delete old file if exists
if os.path.exists(save_path):
    os.remove(save_path)
    print("🗑️ Old dataset deleted.")

# Start collection
for batch_num in range(total_recipes // batch_size):
    print(f"📦 Fetching batch {batch_num + 1} of {total_recipes // batch_size}")
    try:
        recipes_data = get_random_recipes(num_recipes=batch_size, tags=tags)
        if recipes_data:
            processed = process_recipes(recipes_data)
            batch_df = pd.DataFrame(processed)

            if batch_num == 0:
                batch_df.to_csv(save_path, mode='w', index=False, header=True)
            else:
                batch_df.to_csv(save_path, mode='a', index=False, header=False)

            print(f"✅ Batch {batch_num + 1} saved.")
        else:
            print("⚠️ Empty batch received.")
        time.sleep(1)
    except Exception as e:
        print(f"❌ Error in batch {batch_num + 1}: {e}")
        break

# Final file check
if Path(save_path).exists():
    file_size = Path(save_path).stat().st_size / 1024
    print(f"\n🎉 Done. File saved to: {save_path}")
    print(f"📁 File size: {file_size:.1f} KB")
else:
    print("❌ File was not saved.")


In [ ]:
# ✅ STEP 1: Install necessary packages
!pip install -q unsloth datasets bitsandbytes accelerate peft transformers

# ✅ STEP 2: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# ✅ STEP 3: Load model with Unsloth
from unsloth import FastLanguageModel
import torch

adapter_path = "/content/drive/MyDrive/deepseek-7b-recipe-lora2"
model_name = "deepseek-ai/deepseek-llm-7b-base"

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=1024,
    dtype=torch.bfloat16,
    load_in_4bit=True,
    device_map="auto"
)

# ✅ STEP 4: Load LoRA adapters
model.load_adapter(adapter_path)

# ✅ STEP 5: Load and format dataset
import pandas as pd
from datasets import Dataset
import ast

# Load your cleaned CSV with instructions
df = pd.read_csv('/content/drive/MyDrive/spoonacular_recipes_dataset2.csv')

# ✅ Safely format response column
def format_response(row):
    try:
        ingredients = ast.literal_eval(row['ingredients']) if isinstance(row['ingredients'], str) else row['ingredients']
        if not isinstance(ingredients, list):
            ingredients = ["ingredient1", "ingredient2"]
    except:
        ingredients = ["ingredient1", "ingredient2"]

    title = row['title'] if isinstance(row['title'], str) else "Untitled Recipe"
    instructions = row['instructions'] if isinstance(row['instructions'], str) else "Instructions not available."

    return (
        f"{title}\n\n"
        f"Ingredients:\n" +
        "\n".join(f"- {i}" for i in ingredients) +
        "\n\nInstructions:\n" + instructions
    )

df['response'] = df.apply(format_response, axis=1)

# ✅ Format the prompt with proper fallbacks
df['prompt'] = df.apply(
    lambda x: f"<|user|> Generate a recipe using ONLY the following ingredients: {', '.join(ast.literal_eval(x['ingredients'])) if isinstance(x['ingredients'], str) else x['ingredients']}. "
              f"Do not include the following allergens: {x['allergies'] if isinstance(x['allergies'], str) else 'none'}. "
              f"Ensure the recipe follows these dietary preferences: {x['dietary_preferences'] if isinstance(x['dietary_preferences'], str) else 'none'}. "
              f"Include title, ingredients, and detailed step-by-step instructions. <|assistant|>",
    axis=1
)

# ✅ Combine for model input
df['text'] = df['prompt'] + "\n" + df['response']

# ✅ STEP 6: Tokenize
max_length = 1024

def tokenize(example):
    model_inputs = tokenizer(
        example["text"],
        padding="max_length",
        truncation=True,
        max_length=max_length
    )
    model_inputs["labels"] = [
        [-100 if token == tokenizer.pad_token_id else token for token in input_ids]
        for input_ids in model_inputs["input_ids"]
    ]
    return model_inputs

dataset = Dataset.from_pandas(df[['text']])
dataset = dataset.train_test_split(test_size=0.05)

tokenized_train = dataset["train"].map(tokenize, batched=True, remove_columns=["text"])
tokenized_eval = dataset["test"].map(tokenize, batched=True, remove_columns=["text"])

# ✅ STEP 7: Trainer setup
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/deepseek-7b-recipe-lora2-continued2",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    num_train_epochs=2,
    logging_steps=10,
    save_strategy="steps",
    save_steps=250,
    save_total_limit=2,
    bf16=True,
    optim="paged_adamw_8bit",
    warmup_steps=5,
    weight_decay=0.01,
    lr_scheduler_type="linear",
    learning_rate=2e-4,
    report_to="none",
    remove_unused_columns=False
)

model.enable_input_require_grads()

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_eval,
    tokenizer=tokenizer
)

# ✅ STEP 8: Train the model
trainer.train()

# ✅ STEP 9: Save
model.save_pretrained("/content/drive/MyDrive/deepseek-7b-recipe-lora2-continued2")
tokenizer.save_pretrained("/content/drive/MyDrive/deepseek-7b-recipe-lora2-continued2")

print("✅ Fine-tuning complete. Model saved.")


In [ ]:
# ✅ Install required packages
!pip install -q unsloth transformers bitsandbytes accelerate peft

# ✅ Imports and Mount Google Drive
import torch
from unsloth import FastLanguageModel
from transformers import GenerationConfig
from google.colab import drive

drive.mount('/content/drive')

# ✅ Load the fine-tuned model
model_path = "/content/drive/MyDrive/deepseek-7b-recipe-lora2-continued2"

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_path,
    max_seq_length=1024,
    dtype=torch.bfloat16,
    load_in_4bit=True,
    device_map="auto"
)

# ✅ Build prompt dynamically
def build_prompt(ingredients, allergies="", diet=""):
    base = f"<|user|> Generate a recipe using ONLY the following ingredients: {', '.join(ingredients)}. "
    if allergies:
        base += f"Do not include the following allergens: {allergies}. "
    if diet:
        base += f"Ensure the recipe follows these dietary preferences: {diet}. "
    base += "Include title, ingredients, and detailed step-by-step instructions. <|assistant|>"
    return base

# ✅ Generate a recipe with stronger controls
def generate_recipe(prompt, max_tokens=350):
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_new_tokens=max_tokens,
            do_sample=True,
            temperature=0.5,        # More conservative
            top_p=0.8,              # Less diversity
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.pad_token_id
        )

    decoded = tokenizer.decode(output[0], skip_special_tokens=True)

    # ✅ Remove repeated hallucination via manual cutoff
    response = decoded[len(prompt):].strip()
    response = response.split("<|endoftext|>")[0].strip()  # stop if hallucinated
    response = response.split("Ingredients:")[0].strip() + "\nIngredients:\n" + response.split("Ingredients:")[-1].split("Instructions:")[0].strip() + "\n\nInstructions:\n" + response.split("Instructions:")[-1].strip()

    return response

# ✅ Run test cases
test_cases = [
    {
        "ingredients": ["chickpeas", "olive oil", "lemon", "garlic"],
        "allergies": "dairy, gluten",
        "diet": "vegan"
    },
    {
        "ingredients": ["quinoa", "black beans", "bell pepper", "avocado"],
        "allergies": "",
        "diet": "vegetarian"
    },
    {
        "ingredients": ["salmon", "asparagus", "olive oil", "lemon"],
        "allergies": "",
        "diet": ""
    }
]

for i, case in enumerate(test_cases, 1):
    print(f"\n🔹 Test Case {i}")
    prompt = build_prompt(case["ingredients"], case["allergies"], case["diet"])
    output = generate_recipe(prompt)
    print(output)
# might be the best option

In [ ]:
import http.client
import json
import pandas as pd
import os
import time
import re
from pathlib import Path

from google.colab import drive
drive.mount('/content/drive')

RAPIDAPI_KEY = "5bf6609405mshe61b903936f23cep1fd1b6jsn1ecaaf102007"

def get_random_recipes(num_recipes=10, tags=""):
    conn = http.client.HTTPSConnection("spoonacular-recipe-food-nutrition-v1.p.rapidapi.com")
    headers = {
        'x-rapidapi-key': RAPIDAPI_KEY,
        'x-rapidapi-host': "spoonacular-recipe-food-nutrition-v1.p.rapidapi.com"
    }
    url = f"/recipes/random?number={num_recipes}&tags={tags}"
    conn.request("GET", url, headers=headers)
    return json.loads(conn.getresponse().read().decode("utf-8"))

def get_recipe_details(recipe_id):
    conn = http.client.HTTPSConnection("spoonacular-recipe-food-nutrition-v1.p.rapidapi.com")
    headers = {
        'x-rapidapi-key': RAPIDAPI_KEY,
        'x-rapidapi-host': "spoonacular-recipe-food-nutrition-v1.p.rapidapi.com"
    }
    url = f"/recipes/{recipe_id}/information?includeNutrition=false"
    conn.request("GET", url, headers=headers)
    return json.loads(conn.getresponse().read().decode("utf-8"))

def clean_instructions(text):
    if not text or not isinstance(text, str): return ""
    cutoff_phrases = ["fn chefs", "provided by", "visit", "copyright", "donate", "national ms society", "follow me"]
    for phrase in cutoff_phrases:
        idx = text.lower().find(phrase)
        if idx != -1:
            text = text[:idx]
    return re.sub(r"\n{2,}", "\n", text.strip())

def process_recipes(recipes_data):
    recipe_list = []
    for recipe in recipes_data["recipes"]:
        recipe_id = recipe["id"]
        details = get_recipe_details(recipe_id)
        time.sleep(0.5)

        title = details.get("title", "Untitled")
        ingredients = [i["name"] for i in details.get("extendedIngredients", [])]
        raw_steps = [step["step"] for instr in details.get("analyzedInstructions", []) for step in instr.get("steps", [])]
        instructions = clean_instructions("\n".join(raw_steps))

        if len(instructions.split()) < 20: continue

        allergies = []
        if details.get("glutenFree"): allergies.append("gluten free")
        if details.get("dairyFree"): allergies.append("dairy free")
        diet = ', '.join(details.get("diets", [])) or "Not specified"

        recipe_list.append({
            "title": title,
            "ingredients": ingredients if ingredients else ["No ingredients provided"],
            "instructions": instructions,
            "allergies": ', '.join(allergies) if allergies else "Not specified",
            "dietary_preferences": diet
        })

    return recipe_list

# CONFIG
total_recipes = 2000
batch_size = 10
save_path = "/content/drive/MyDrive/spoonacular_recipes_dataset_v3.csv"

os.makedirs(os.path.dirname(save_path), exist_ok=True)
if os.path.exists(save_path): os.remove(save_path)

for batch_num in range(total_recipes // batch_size):
    print(f"📦 Batch {batch_num + 1}")
    try:
        data = get_random_recipes(batch_size)
        processed = process_recipes(data)
        if processed:
            df = pd.DataFrame(processed)
            df.to_csv(save_path, mode='a', index=False, header=not os.path.exists(save_path))
            print(f"✅ Saved {len(df)} entries.")
        time.sleep(1)
    except Exception as e:
        print(f"❌ Error in batch {batch_num + 1}: {e}")
        break

print(f"\n🎉 Dataset complete. Saved to: {save_path}")


In [ ]:
!pip install -q unsloth datasets bitsandbytes accelerate peft transformers

from google.colab import drive
drive.mount('/content/drive')

from unsloth import FastLanguageModel
import torch
import pandas as pd
from datasets import Dataset
import ast

# ✅ Load base model + adapter (lora2)
adapter_path = "/content/drive/MyDrive/deepseek-7b-recipe-lora2"
model_name = "deepseek-ai/deepseek-llm-7b-base"

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=1024,
    dtype=torch.bfloat16,
    load_in_4bit=True,
    device_map="auto"
)
model.load_adapter(adapter_path)

# ✅ Load v3 dataset
df = pd.read_csv("/content/drive/MyDrive/spoonacular_recipes_dataset_v3.csv")

def format_response(row):
    try:
        ingredients = ast.literal_eval(row['ingredients']) if isinstance(row['ingredients'], str) else row['ingredients']
    except:
        ingredients = ["ingredient1", "ingredient2"]
    title = row.get('title', 'Untitled Recipe')
    instructions = row.get('instructions', 'Instructions not available.')
    return f"{title}\n\nIngredients:\n" + "\n".join(f"- {i}" for i in ingredients) + "\n\nInstructions:\n" + instructions

df['response'] = df.apply(format_response, axis=1)
df['prompt'] = df.apply(
    lambda x: f"<|user|> Generate a recipe using ONLY the following ingredients: {', '.join(ast.literal_eval(x['ingredients'])) if isinstance(x['ingredients'], str) else x['ingredients']}. " +
              (f"Do not include the following allergens: {x['allergies']}. " if isinstance(x['allergies'], str) and x['allergies'] != "Not specified" else "") +
              (f"Ensure the recipe follows these dietary preferences: {x['dietary_preferences']}. " if isinstance(x['dietary_preferences'], str) and x['dietary_preferences'] != "Not specified" else "") +
              "Include title, ingredients, and detailed step-by-step instructions. <|assistant|>",
    axis=1
)
df["text"] = df["prompt"] + "\n" + df["response"] + "\n<|endoftext|>"

# ✅ Tokenization
max_length = 1024
def tokenize(example):
    tokens = tokenizer(example["text"], padding="max_length", truncation=True, max_length=max_length)
    tokens["labels"] = [[-100 if token == tokenizer.pad_token_id else token for token in ids] for ids in tokens["input_ids"]]
    return tokens

dataset = Dataset.from_pandas(df[["text"]]).train_test_split(test_size=0.05)
tokenized_train = dataset["train"].map(tokenize, batched=True, remove_columns=["text"])
tokenized_eval = dataset["test"].map(tokenize, batched=True, remove_columns=["text"])

# ✅ Training setup
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/deepseek-7b-recipe-lora2-v3",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    num_train_epochs=1,
    logging_steps=10,
    save_strategy="steps",
    save_steps=250,
    save_total_limit=2,
    bf16=True,
    optim="paged_adamw_8bit",
    warmup_steps=5,
    weight_decay=0.01,
    learning_rate=2e-4,
    lr_scheduler_type="linear",
    report_to="none",
    remove_unused_columns=False
)

model.enable_input_require_grads()

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_eval,
    tokenizer=tokenizer
)

trainer.train()

# ✅ Save model
model.save_pretrained("/content/drive/MyDrive/deepseek-7b-recipe-lora2-v3")
tokenizer.save_pretrained("/content/drive/MyDrive/deepseek-7b-recipe-lora2-v3")
print("✅ Training complete. Saved to lora2-v3.")


In [ ]:
from transformers import pipeline
import torch
from unsloth import FastLanguageModel

# Load your model and tokenizer (assuming these are already done)
model_path = "/content/drive/MyDrive/deepseek-7b-recipe-lora2-v3"
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_path,
    max_seq_length=1024,
    dtype=torch.bfloat16,
    load_in_4bit=True,
    device_map="auto"
)

# Inference function using greedy decoding (deterministic output)
def generate_recipe(prompt):
    # Use greedy decoding for inference (no randomness)
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    output = model.generate(inputs["input_ids"], max_length=512, do_sample=False, temperature=0.7)
    return tokenizer.decode(output[0], skip_special_tokens=True)

# Inference function using sampling (randomized output)
def generate_recipe_with_sampling(prompt):
    # Use sampling for inference (if you'd like variability)
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    output = model.generate(inputs["input_ids"], max_length=512, do_sample=True, top_p=0.95, temperature=0.7)
    return tokenizer.decode(output[0], skip_special_tokens=True)

# Test Cases for Recipe Generation

test_cases = [
    {
        'input_prompt': "Generate a recipe using ONLY the following ingredients: carrot, chicken, rice, garlic, olive oil. Do not include the following allergens: peanut. Ensure the recipe follows these dietary preferences: gluten free, dairy free. Include title, ingredients, and detailed step-by-step instructions."
    },
    {
        'input_prompt': "Generate a recipe using ONLY the following ingredients: tomatoes, basil, mozzarella. Do not include the following allergens: gluten. Ensure the recipe follows these dietary preferences: vegetarian, gluten free. Include title, ingredients, and detailed step-by-step instructions."
    },
    {
        'input_prompt': "Generate a recipe using ONLY the following ingredients: spinach, feta cheese, olive oil, lemon. Do not include the following allergens: dairy. Ensure the recipe follows these dietary preferences: vegan. Include title, ingredients, and detailed step-by-step instructions."
    }
]

# Running the test cases and generating recipes

for i, test_case in enumerate(test_cases, start=1):
    print(f"\nTest Case {i} (Greedy Decoding):")
    print("=" * 50)
    generated_recipe = generate_recipe(test_case['input_prompt'])
    print(generated_recipe)

    print("\nTest Case {i} (Sampling Decoding):")
    print("=" * 50)
    generated_recipe_sampling = generate_recipe_with_sampling(test_case['input_prompt'])
    print(generated_recipe_sampling)
    print("=" * 50)
